In [3]:
import json
import time
import os
import requests

import fml_manager
from fml_manager import *

manager = fml_manager.FMLManager()

In [6]:
response = manager.load_data(url='./data/breast_b.csv', namespace='fate_flow_test_breast', table_name='breast_b', work_mode=1, head=1, partition=10)
output = json.loads(response.content)
print(output)
guest_job_id = output["jobId"]
guest_query_condition = {
    'job_id':guest_job_id
}

manager.query_job_status(guest_query_condition)

{'data': {'board_url': 'http://fateboard:8080/index.html#/dashboard?job_id=2020063006505834637941&role=local&party_id=0', 'job_dsl_path': '/data/projects/fate/python/jobs/2020063006505834637941/job_dsl.json', 'job_runtime_conf_path': '/data/projects/fate/python/jobs/2020063006505834637941/job_runtime_conf.json', 'logs_directory': '/data/projects/fate/python/logs/2020063006505834637941', 'namespace': 'fate_flow_test_breast', 'table_name': 'breast_b'}, 'jobId': '2020063006505834637941', 'retcode': 0, 'retmsg': 'success'}
Status: waiting
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: success


'success'

In [9]:
# Pipline
data_io = ComponentBuilder()\
    .with_name('dataio_0')\
    .with_module('DataIO')\
    .with_input_data('arg.train_data')\
    .with_output_data('train')\
    .with_output_model('dataio')\
    .with_need_deploy(True).build()
        
hetero_feature_binning = ComponentBuilder()\
    .with_name('hetero_feature_binning_0')\
    .with_module('HeteroFeatureBinning')\
    .with_input_data('dataio_0.train')\
    .with_output_data('train')\
    .with_output_model("hetero_feature_binning").build()

hetero_feature_selection = ComponentBuilder()\
    .with_name('hetero_feature_selection_0')\
    .with_module('HeteroFeatureSelection')\
    .with_input_data('hetero_feature_binning_0.train')\
    .with_output_data('train')\
    .with_output_model('selected')\
    .with_input_isometric_model('hetero_feature_binning_0.hetero_feature_binning').build()

hetero_lr = ComponentBuilder()\
    .with_name('hetero_lr_0')\
    .with_module('HeteroLR')\
    .with_input_train_data('hetero_feature_selection_0.train')\
    .with_output_data('train')\
    .with_output_model('hetero_lr').build()

evaluation = ComponentBuilder()\
    .with_name('evaluation_0')\
    .with_module('Evaluation')\
    .with_input_data('hetero_lr_0.train')\
    .with_output_data('evaluate').build()

pipline = PiplineBuilder()\
    .with_components(
        data_io, 
        hetero_feature_selection,  
        hetero_feature_binning, 
        hetero_lr, 
        evaluation).build()



# Configuration
initiator = InitiatorBuilder()\
    .with_role("guest")\
    .with_party_id(10000).build()


job_parameters = JobParametersBuilder()\
    .with_work_mode(1).build()

role = RoleBuilder()\
    .with_guest(10000)\
    .with_host(9999)\
    .with_arbiter(9999).build()

guest_data_io_config = {
    "with_label": [True],
    "label_name": ["y"],
    "label_type": ["int"],
    "output_format": ["dense"]
}

host_data_io_config = {
    "with_label": [False],
    "output_format": ["dense"]
}

role_parameters = RoleParametersBuilder()\
    .with_guest_train_data(namespaces=['fate_flow_test_breast'], names=['brest_b'])\
    .with_guest_module_config(modules=['dataio_0'], configs=[guest_data_io_config])\
    .with_host_train_data(namespaces=['fate_flow_test_breast'], names=['brest_a'])\
    .with_host_module_config(modules=['dataio_0'], configs=[host_data_io_config]).build()


hetero_lr_params = {
    "penalty": "L2",
    "optimizer": "rmsprop",
    "eps": 1e-5,
    "alpha": 0.01,
    "max_iter": 3,
    "converge_func": "diff",
    "batch_size": 320,
    "learning_rate": 0.15,
    "init_param": {
        "init_method": "random_uniform"
    }
}

algorithm_parameters = AlgorithmParametersBuilder()\
    .with_module_config(modules=['hetero_lr_0'], configs=[hetero_lr_params]).build()

config = ConfigBuilder()\
    .with_initiator(initiator)\
    .with_job_parameters(job_parameters)\
    .with_role(role)\
    .with_role_parameters(role_parameters)\
    .with_algorithm_parameters(algorithm_parameters).build()


In [10]:
response = manager.submit_job(pipline.to_dict(),config.to_dict())
manager.prettify(response, verbose=True)
stdout = json.loads(response.content)
jobId = stdout["jobId"]
query_condition = {
    'job_id':jobId
}

model_id, model_version = '', ''
manager.query_job_status(query_condition)

manager.prettify(response, verbose=True)
output = json.loads(response.content)
model_id, model_version = output["data"]["model_info"]["model_id"], output["data"]["model_info"]["model_version"]

Success!
{
    "data": {
        "board_url": "http://fateboard:8080/index.html#/dashboard?job_id=2020063006531172788744&role=guest&party_id=9999",
        "job_dsl_path": "/data/projects/fate/python/jobs/2020063006531172788744/job_dsl.json",
        "job_runtime_conf_path": "/data/projects/fate/python/jobs/2020063006531172788744/job_runtime_conf.json",
        "logs_directory": "/data/projects/fate/python/logs/2020063006531172788744",
        "model_info": {
            "model_id": "arbiter-9999#guest-9999#host-9999#model",
            "model_version": "2020063006531172788744"
        }
    },
    "jobId": "2020063006531172788744",
    "retcode": 0,
    "retmsg": "success"
}
Status: waiting
Status: waiting
Status: waiting
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: run

In [21]:


# Pipline
data_io = ComponentBuilder()\
    .with_name('dataio_0')\
    .with_module('DataIO')\
    .with_input_data('arg.train_data')\
    .with_output_data('train')\
    .with_output_model('dataio')\
    .with_need_deploy(True).build()
        
hetero_feature_binning = ComponentBuilder()\
    .with_name('hetero_feature_binning_0')\
    .with_module('HeteroFeatureBinning')\
    .with_input_data('dataio_0.train')\
    .with_output_data('train')\
    .with_output_model("hetero_feature_binning").build()

hetero_feature_selection = ComponentBuilder()\
    .with_name('hetero_feature_selection_0')\
    .with_module('HeteroFeatureSelection')\
    .with_input_data('hetero_feature_binning_0.train')\
    .with_output_data('eval')\
    .with_output_model('selected')\
    .with_input_isometric_model('hetero_feature_binning_0.hetero_feature_binning').build()


evaluation = ComponentBuilder()\
    .with_name('evaluation_0')\
    .with_module('Evaluation')\
    .with_input_data('hetero_feature_selection_0.eval')\
    .with_output_data('evaluate').build()

pipline = PiplineBuilder()\
    .with_components(
        data_io, 
        hetero_feature_selection,  
        hetero_feature_binning, 
        evaluation).build()


# Configuration
initiator = InitiatorBuilder()\
    .with_role("guest")\
    .with_party_id(10000).build()


job_parameters = JobParametersBuilder()\
    .with_work_mode(1)\
    .with_job_type("predict")\
    .with_model_id("arbiter-9999#guest-10000#host-9999#model")\
    .with_model_version("2020060802475836992436").build()

role = RoleBuilder()\
    .with_guest(10000)\
    .with_host(9999)\
    .with_arbiter(9999).build()


role_parameters = RoleParametersBuilder()\
    .with_guest_eval_data(namespaces=['fate_flow_test_breast'], names=['brest_b'])\
    .with_host_eval_data(namespaces=['fate_flow_test_breast'], names=['breast_a']).build()




config = ConfigBuilder()\
    .with_initiator(initiator)\
    .with_job_parameters(job_parameters)\
    .with_role(role)\
    .with_role_parameters(role_parameters).build()




In [22]:
response = manager.submit_job(pipline.to_dict(), config.to_dict())
manager.prettify(response, verbose=True)
stdout = json.loads(response.content)
jobId = stdout["jobId"]
query_condition = {
    'job_id': jobId
}

Success!
{
    "data": {
        "board_url": "http://fateboard:8080/index.html#/dashboard?job_id=2020063006573868964950&role=guest&party_id=9999",
        "job_dsl_path": "/data/projects/fate/python/jobs/2020063006573868964950/job_dsl.json",
        "job_runtime_conf_path": "/data/projects/fate/python/jobs/2020063006573868964950/job_runtime_conf.json",
        "logs_directory": "/data/projects/fate/python/logs/2020063006573868964950",
        "model_info": {
            "model_id": "arbiter-9999#guest-9999#host-9999#model",
            "model_version": "2020063006531172788744"
        }
    },
    "jobId": "2020063006573868964950",
    "retcode": 0,
    "retmsg": "success"
}


In [23]:
print(query_condition)

{'job_id': '2020063006573868964950'}


In [24]:
model_id, model_version = '', ''
manager.query_job_status(query_condition)

manager.prettify(response, verbose=True)
output = json.loads(response.content)
model_id, model_version = output["data"]["model_info"]["model_id"], output["data"]["model_info"]["model_version"]

Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: 

In [25]:
manager.track_component_output_data(job_id='2020060806580950942753', role='guest', party_id='10000', component_name='hetero_lr_0')

Success!
{
    "data": [
        [
            "8",
            0,
            1,
            0.7046115406302281,
            {
                "0": 0.29538845936977187,
                "1": 0.7046115406302281
            },
            "predict"
        ],
        [
            "17",
            0,
            1,
            0.9564225652538682,
            {
                "0": 0.04357743474613185,
                "1": 0.9564225652538682
            },
            "predict"
        ],
        [
            "26",
            0,
            1,
            0.9661537574810894,
            {
                "0": 0.03384624251891055,
                "1": 0.9661537574810894
            },
            "predict"
        ],
        [
            "35",
            0,
            0,
            0.19285719929095904,
            {
                "0": 0.807142800709041,
                "1": 0.19285719929095904
            },
            "predict"
        ],
        [
            "44",
            

<Response [200]>